# Reading data into Julia

This notebooks demonstrates reading data into Julia and performing basic manipulations.

## Load libraries

First, we need to load the libraries we will be working with. In Julia, functionality tends to be split between libraries much more than it is in other languages such as Python or R. To read data from a CSV file, we need two libraries: CSV and DataFrames. We also load the StatsBase package, which contains many basic statistical functions (mean, standard deviation, etc.). DataFramesMeta provides some additional syntax to make analysis more readable.

In [ ]:
using CSV, DataFrames, StatsBase, DataFramesMeta

## Reading data

Since CSV reading and tabular data manipulation are split across libraries, we use the `CSV.read` function to load data into a `DataFrame`.

Unlike many other languages, Julia in an interactive session will print out the result of an operation, even if you assign it to a variable. To prevent this, add a semicolon at the end of the last line.

In [ ]:
sensors = CSV.read("data/bay_area_freeways.csv", DataFrame)

## Accessing data in a loaded table

Once the data is loaded, we can access it by column or by row. Each column is a Vector (known as an array or list in other languages).

In [ ]:
sensors.avg_occ

## Computing functions of columns

Many functions in Julia are defined for Vectors of numbers—for instance, the mean.

In [ ]:
mean(sensors.avg_occ)

## Dealing with missing data

Datasets often have missing values. Julia represents these with the special value `missing`. Math with `missing` will generally result in `missing`, to avoid confusion when you don't use all the data you think you should. You can remove the missing values with `skipmissing`, and find them with `ismissing`.

In [ ]:
sum(ismissing.(sensors.avg_occ))

In [ ]:
mean(skipmissing(sensors.avg_occ))

## Transforming columns

You can apply any Julia function elementwise using the . operator.

In [ ]:
log.(sensors.avg_occ)

## Filtering data

You can also filter by rows by using logical operators. Like functions, they can be applied elementwise with a . operator, but the . goes before the operator. We have to explicitly specify that we want all columns (`:`)

In [ ]:
sensors[sensors.avg_occ .> 0.05,:]

## Missing data, again

We get an error with the above command, because of the missing value, again. Julia doesn't know whether the missing value was greater than 0.05. We can use the `coalesce` function to make an assumption that it was not. `coalesce` returns its first nonmissing argument. In this case, we will use it with a `.` operator to apply to the entire array.

In [ ]:
sensors[coalesce.(sensors.avg_occ .> 0.05, false),:]

## Removing missing data

Alternately, we can just remove the missing data. This is pretty easy to do with the `ismissing` function and the filtering we've seen above.

In [ ]:
sensors = sensors[.!ismissing.(sensors.avg_occ),:]

## Assertions

It's a good idea to add assertions within your code regularly. You write out a check that should be true, and if it is not an error will be printed.

In [ ]:
# this will print no output if there are no missing sensors
@assert !any(ismissing.(sensors.avg_occ))

In [ ]:
# but if the assertion fails, you'll get an error
@assert all(sensors.avg_occ .< 0.05)

### Logical operators

`sensors.avg_occ .> 0.05` is nothing particularly special, it's just creating a vector of boolean values, one for each row of the data, indicating whether that row has `avg_occ` > 0.05.

In [ ]:
sensors.avg_occ .> 0.05

## Split-apply-combine

Split-apply-combine is a common pattern in many other data manipulation languages (e.g. groupby in Python, group_by %>% summarize in R). This pattern involves dividing the dataset based on the values of certain variables, applying operations to each subset, and combining the results - most often into a single row per group. For instance, we can compute mean occupancy by freeway and direction, below.

In Julia, we use the `groupby` and `combine` functions for this. We reference variable names by placing a : in front of them, or putting them in double quotes if they contain spaces, dashes, or start with a number. There are multiple ways to write out this operation; this uses plain Julia syntax to pass the result of the groupby into the combine operation.

In [ ]:
combine(
    # Group the data by these variables. You only need [] if you are grouping by multiple variables.
    groupby(sensors, [:freeway_number, :direction]),
    # and create a new variable mean_avg_occ by applying the mean() function to the avg_occ column in
    # each subset of the data
    :avg_occ => mean => :mean_avg_occ
)

A more readable way to write this is as a "chain," where we write out multiple function calls in order, and the result of each function call is used as the first argument to the next. If you've used `tidyverse` in R this will be familiar. This code does exactly the same thing as the code in the previous cell.

In [ ]:
@chain begin
    # Group the data by these variables. You only need [] if you are grouping by multiple variables.
    groupby(sensors, [:freeway_number, :direction])
    # and create a new variable mean_avg_occ by applying the mean() function to the avg_occ column in
    # each subset of the data
    combine(:avg_occ => mean => :mean_avg_occ)
end

## Joins

A very common operation is to join data sources together using a common column. In Julia, we can use the `leftjoin` function to do this (other types of joins are available as well). We will add sensor metadata to the database.

In [ ]:
meta = CSV.read("data/sensor_meta.csv", DataFrame)

In [ ]:
sensors = leftjoin(sensors, meta, on=:station=>:ID)

In [ ]:
sensors.Lanes

## Shut down the kernel!

When done, choose Kernel -> Shut Down Kernel to free memory for the next exercise.